In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle
import os
import warnings
warnings.filterwarnings("ignore")

# Basic text cleaning function
def clean_text(text):
    if pd.isna(text):
        return "missing"
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.lower().strip()  # Convert to lowercase and strip whitespace
    return text

def clean_text(text):
    # Check if the input is a string before applying regex
    if not isinstance(text, str):
        return "missing"  # Return a default value for non-string input (e.g., NaN, numbers)

    # Now apply your cleaning logic
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.lower().strip()  # Convert to lowercase and strip whitespace
    return text

def extract_tables(file_path, sheet_name, check_names, check_idx, start_col, end_col):
    """
    Extract subtables from a sheet using pandas DataFrame.
    Subtables are separated by empty rows.
    Only subtables where the header in index 2 (column 3) contains 'Movimiento', 
    'Cta Cte USD', 'TC Nacional', or 'TC Internacional' will be added to the list of subtables.
    """

    # Read the Excel sheet into a DataFrame (only the first `col_num` columns)
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=range(start_col, end_col), engine="openpyxl", header=None)
    
    # Replace all-NaN rows with a marker
    df["is_empty"] = df.isnull().all(axis=1)
    
    # Split the DataFrame into subtables based on empty rows
    subtables = []
    current_table = []

    for _, row in df.iterrows():
        if row["is_empty"]:
            # If we encounter an empty row, save the current table (if not empty)
            if current_table:
                # Convert current table to DataFrame
                table = pd.DataFrame(current_table).reset_index(drop=True)

                # Find the row where the column index 2 matches one of the check_names
                header_row_index = None
                for i, row in table.iterrows():
                    value = str(row[check_idx]).strip()
                    if value in check_names:
                        header_row_index = i
                        break

                if header_row_index is not None:
                    # Set the identified row as the header and drop all rows above it
                    table.columns = table.iloc[header_row_index]  # Set the header
                    table = table[header_row_index + 1:].reset_index(drop=True)  # Drop rows above the header

                    # Append the table if it's valid (it has content after setting the header)
                    if not table.empty:
                        subtables.append(table)

                current_table = []  # Reset for the next table
        else:
            # Add non-empty rows to the current table
            current_table.append(row[:-1])  # Exclude the "is_empty" column

    # Add the last table if present
    if current_table:
        # Convert current table to DataFrame
        table = pd.DataFrame(current_table).reset_index(drop=True)

        # Find the row where the column index 2 matches one of the check_names
        header_row_index = None
        for i, row in table.iterrows():
            value = str(row[check_idx]).strip()
            if value in check_names:
                header_row_index = i
                break

        if header_row_index is not None:
            # Set the identified row as the header and drop all rows above it
            table.columns = table.iloc[header_row_index]  # Set the header
            table = table[header_row_index + 1:].reset_index(drop=True)  # Drop rows above the header

            # Append the table if it's valid (it has content after setting the header)
            if not table.empty:
                subtables.append(table)

    # Remove subtables where the first column is only NaN
    subtables = [
        table for table in subtables 
        if not table.iloc[:, 0].isna().all()  # Check if the first column is NOT all NaN
    ]

    # Ensure unique column names for all subtables at once
    for idx, table in enumerate(subtables):
        # Use pandas' built-in method to ensure unique column names
        table.columns = [
            f"Unnamed_{i}" if pd.isna(col) else col for i, col in enumerate(table.columns)
        ]

    return subtables

def training_model(labeled_data, label_col_idx):
    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(labeled_data['Combined_Text'], labeled_data.iloc[:, label_col_idx], test_size=0.2, random_state=42)
    # TF-IDF vectorization
    vectorizer = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
    X_train_tfidf = vectorizer.fit_transform(X_train)
    # Train the classifier
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train_tfidf, y_train)
    return model, vectorizer, X_train, X_test, y_test

def classify_text(model, vectorizer, *args):
    # Combine and clean input text
    combined_text = ' '.join(clean_text(arg) for arg in args)
    text_tfidf = vectorizer.transform([combined_text])  # Transform using the trained vectorizer
    # Get predicted label and probabilities
    label = model.predict(text_tfidf)[0]  # Predicted label
    probabilities = model.predict_proba(text_tfidf)[0]  # Probabilities for all classes
    # Get the confidence score for the predicted label
    confidence_score = max(probabilities)  # Highest probability corresponds to the predicted label
    return label, confidence_score

# # Para Security leer desde el 0 hasta el 7
# # Check index 1

# # BCI 1 a 11
# # Check index 3

# # Example usage
# file_path = "./EERRs/2018/EERR Ene 18 rev0.xlsx"
# sheet_name = "BCI "
# check_names = ['Movimiento', 'Cta Cte USD', 'TC Nacional', 'TC Internacional']
# check_idx = 3
# #check_names = ['Descripción']
# end_col = 7
# start_col = 0
# tables = extract_tables(file_path, sheet_name, check_names, check_idx, start_col, end_col)
# for i, table in enumerate(tables):
#     print(table.iloc[:, 6].to_string())
#     print()


## Entrenamiento del modelo

**Planilla BCI**

In [ ]:
directory = "./EERRs"
sheet_name = "BCI "
tabla_dict = {'C85' : 0, 'CUSD' : 1, 'TCN' : 2, 'TCI' : 3}
tabla = 'TCI'
indx_tab = tabla_dict[tabla]

label_col_indx = 7
check_names = ['Movimiento', 'Cta Cte USD', 'TC Nacional', 'TC Internacional']
check_idx = 3
start_col, end_col = 1, 11

df_list = []
for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".xlsx"):  # Only process .xlsx files
            file_path = os.path.join(root, filename)  # Get the full file path
            subtables = extract_tables(file_path, sheet_name, check_names, check_idx, start_col, end_col)  # Assuming `extract_tables` is defined
            df = subtables[indx_tab]
            # print(file_path)
            # print(df.to_string())
            # print()
            df_list.append(df)  # Append the dataframe to the list
data = pd.concat(df_list, ignore_index=True)
labeled_data = data.dropna(subset=[data.columns[label_col_indx]])
#print(labeled_data.to_string())

# Procesar texto ---
# Procesar texto ---
feature_index_list = [2, 8, 9] # Index de las columnas que contienen el feature para entrenar al modelo. En este caso Movimiento, Fact, Prove, etc.
#feature_index_list = [2] # Para CUSD, TCN, TCI

# Clean the text columns

# Clean the specified columns
for col_index in feature_index_list:
    labeled_data.iloc[:, col_index] = labeled_data.iloc[:, col_index].apply(clean_text)

# Combine the specified columns into a single feature
labeled_data['Combined_Text'] = labeled_data.iloc[:, feature_index_list].agg(' '.join, axis=1)

model, vectorizer, X_train, X_test, y_test = training_model(labeled_data, label_col_idx=label_col_indx)

# Evaluación del modelo
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

# --- SAVE THE MODEL AND VECTORIZER ---
with open(f"./modelo_{tabla}.pkl", "wb") as model_file, open(f"./vectorizer_{tabla}.pkl", "wb") as vectorizer_file:
    pickle.dump(model, model_file)  # Save the model
    pickle.dump(vectorizer, vectorizer_file)  # Save the vectorizer

#print(labeled_data.to_string())

                        precision    recall  f1-score   support

      Comisión Expedia       0.00      0.00      0.00         1
        Comisión otras       0.91      1.00      0.95        30
Gastos Admininstración       0.00      0.00      0.00         2

              accuracy                           0.91        33
             macro avg       0.30      0.33      0.32        33
          weighted avg       0.83      0.91      0.87        33



**Planilla Security**

In [61]:
directory = "./EERRs"
sheet_name = "Security" 
indx_tab = 0 

# Security 0, 7
# Check index 1
# Security index Calve: 6
label_col_indx = 6
check_names = ['Descripción']
check_idx = 1
start_col, end_col = 0, 7

df_list = []
for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".xlsx"):  # Only process .xlsx files
            file_path = os.path.join(root, filename)  # Get the full file path
            subtables = extract_tables(file_path, sheet_name, check_names, check_idx, start_col, end_col)  # Assuming `extract_tables` is defined
            df = subtables[indx_tab]
            df_list.append(df)  # Append the dataframe to the list
data = pd.concat(df_list, ignore_index=True)
labeled_data = data.dropna(subset=[data.columns[label_col_indx]])

# Procesar texto ---
feature_index_list = [1]
# Clean the text columns
# Clean the specified columns
for col_index in feature_index_list:
    labeled_data.iloc[:, col_index] = labeled_data.iloc[:, col_index].apply(clean_text)

# Combine the specified columns into a single feature
labeled_data['Combined_Text'] = labeled_data.iloc[:, feature_index_list].agg(' '.join, axis=1)

model, vectorizer, X_train, X_test, y_test = training_model(labeled_data, label_col_idx=label_col_indx)

# Evaluación del modelo
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

# --- SAVE THE MODEL AND VECTORIZER ---
with open("./modelo_security.pkl", "wb") as model_file, open("./vectorizer_security.pkl", "wb") as vectorizer_file:
    pickle.dump(model, model_file)  # Save the model
    pickle.dump(vectorizer, vectorizer_file)  # Save the vectorizer

                                 precision    recall  f1-score   support

                   Comisión Bco       1.00      1.00      1.00        15
               Gastos Operación       0.00      0.00      0.00         2
                     Goperación       0.00      0.00      0.00         1
                      Impuestos       1.00      0.86      0.92        14
                       Prestamo       1.00      1.00      1.00         3
                        Retiros       0.80      1.00      0.89         8
                        Sueldos       1.00      0.94      0.97        18
Transbank deposito ventanilla $       0.79      0.92      0.85        12
                 Transf Egresos       0.98      1.00      0.99        42
                 Transf Ingreso       0.89      0.98      0.93       142
              Ventas deposito $       1.00      0.24      0.38        17

                       accuracy                           0.92       274
                      macro avg       0.77      0

## Cargar modelo

In [ ]:
# --- LOAD THE MODEL AND VECTORIZER ---
with open("logistic_model.pkl", "rb") as model_file, open("tfidf_vectorizer.pkl", "rb") as vectorizer_file:
    loaded_clf = pickle.load(model_file)  # Load the model
    loaded_vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer

In [60]:
mov = ' Cargo Cuenta BCI'
fact = 'Fact 2499,2753,2587,2695'
prove = 'MIGUEL ANGEL NIEVAS                          '
label, score = classify_text(model, vectorizer, mov, fact, prove)
print(type(score))

<class 'numpy.float64'>
